## 문제1

`https://data.hossam.kr/py/eng.txt` 
 파파고 API를 활용하여 위 사이트에서 텍스트를 가져와서 번역한 후 
 
 원문 | 번역 
 |--|--|


 내용이 정리된 csv파일 저장

In [6]:
import requests
import json
import os
from dotenv import load_dotenv


url = 'https://data.hossam.kr/py/eng.txt'
session = requests.Session()
session.headers.update({
    "Referer" : "", 
    "User_Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36" 
})
try: 
    r = session.get(url)
    if r.status_code != 200:
        msg = f'[{r.status_code} Error] {r.reason} 에러가 발생함'
        raise Exception(msg)
except Exception as e :
    print('접속에 실패했습니다.', e, sep = '\n')
r.encoding = 'utf-8' # 읽은 후 한글이 깨지면 euc-kr로 변경

A = r.text

source	= 'en' # 번역할 원문의 언어코드
target = 'ko' # 번역될 문장의 언어코드


# 네이버에서 발급받은 연동키
load_dotenv(dotenv_path = '../../../dot_env/.env')
clientId = os.environ.get("naverId")
clientSecret = os.environ.get("naverPw")
url = 'https://openapi.naver.com/v1/papago/n2mt'


session = requests.Session()
session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "X-Naver-Client-Id" : clientId,
    "X-Naver-Client-Secret" : clientSecret
})

try:
    r = session.post(url, data ={
        "source" : source,
        "target" : target,
        "text" : A
    })
    
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
except Exception as e:
    print("접속에 실패했습니다.")
    print(e)

r.encoding = "utf-8"
mydict = json.loads(r.text)
B = mydict['message']['result']['translatedText']
A_ = A.split('\n')
B_ = B.split('\n')
with open('연구과제1.csv','w',encoding='euc-kr') as f:
    f.write('원문, 번역본 \n')
    for i in range(len(A_)):
        f.write('"%s","%s"\n' %(A_[i], B_[i]))  # 내용안에 ,가 있는 경우 csv파일에서 다른행으로 인식된다. 따라서 ""로 묶어주면 ""안에 ,가 있는것으로 인식하여 같은행으로 인식시킬 수 있다.
print("fin :)")


fin :)


A를 애초에 리스트로 쪼개서 반복문을 돌려서 번역

=> 
```python
A = r.text.split(',')

B = []
for i in A:
    try:
        r = session.post(url, data ={
            "source" : source,
            "target" : target,
            "text" : i
        })
    
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

    r.encoding = "utf-8"
    mydict = json.loads(r.text)
    B.append(mydict['message']['result']['translatedText'])
```

## 문제 2

`https://data.hossam.kr/py/bus_station.csv` 파일에는 서울시의 버스정류장에 대한 위도와 경도가 포함되어 있습니다. 이 정보를 활용하여 각 버스정류장의 주소를 조회후 CSV 파일을 생성하세요.

데이터 수집은 비동기 방식으로 최대 50개까지 동시에 처리하세요.

CSV 파일의 형식은 아래와 같습니다.

| 정류소번호 | 정류소명 | X좌표 | Y좌표 | 정류소 타입 | 우편번호 | 주소 |
|---|---|---|---|---|---|---|
| 1001 | 종로2가사거리 | 126.987752 | 37.569808 | 중앙차로 | XXXXX | 서울특별시 종로구 ... |

이 문제를 해결하기 위해 `https://www.vworld.kr/dev/v4api.do` 사이트에서 OpenAPI에 대한 키를 직접 발급받고 API 연동 규격을 파악해야 합니다.

In [ ]:
import requests
import json
import datetime as dt
import os
import concurrent.futures as futures

url = 'https://data.hossam.kr/py/bus_station.csv'
session = requests.Session()

session.headers.update({
    "Referer" : "",
    "User_Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
})
try: 
    r = session.get(url)
    if r.status_code != 200:
        msg = f'[{r.status_code} Error] {r.reason} 에러가 발생함'
        raise Exception(msg)
except Exception as e :# 사이트 주소가 잘못된 경우 에러발생 // 사이트주소만 올바르고 그 이후가 잘못된 경우(파일명이 잘못된 경우)는 파이썬이 감지못함
    print('접속에 실패했습니다.', e, sep = '\n')
r.encoding = 'euc-kr' # 읽은 후 한글이 깨지면 euc-kr로 변경
# 정류소번호,	정류소명,	X좌표,	Y좌표,	정류소 타입  + 우편번호 + 주소
# print(r.text)
A = r.text
B = A.split('\n')



def SetZipcodeandAddress(i,B,a,b,session):
        url = 'https://api.vworld.kr/req/address?service=address&request=getAddress&key=DB986055-D8D7-3F8F-9BBE-3F4E00712330&point=%0.9f,%0.9f&type=both' % (a,b)
        try: 
            r = session.get(url)
            if r.status_code != 200:
                msg = f'[{r.status_code} Error] {r.reason} 에러가 발생함'
                raise Exception(msg)
        
            r.encoding = 'utf-8' # 읽은 후 한글이 깨지면 euc-kr로 변경
            mydict = json.loads(r.text)
            
            B[i] += ','+(mydict['response']['result'][0]['zipcode'] or 'Null')+','
            B[i] += mydict['response']['result'][0]['text']+'\n'
        except Exception as e :# 사이트 주소가 잘못된 경우 에러발생 // 사이트주소만 올바르고 그 이후가 잘못된 경우(파일명이 잘못된 경우)는 파이썬이 감지못함
            print('접속에 실패했습니다.', e, sep = '\n')
            


fname = dt.datetime.now().strftime('주소조회_%y%m%d_%H%M%S.csv')
with futures.ThreadPoolExecutor(max_workers=50) as executor:
    for i, v in enumerate(B[1:]):
        if v == '':break
        C = v.split(',')
        try:
            a = float(C[2])
            b = float(C[3])
        except:
            a = float(C[3])
            b = float(C[4])
        executor.submit(SetZipcodeandAddress,i,B,a,b,session)

with open(fname,'w',encoding='utf-8') as f:
    f.write(f'{B[0]}, 우편번호, 주소\n')
    for i, v in enumerate(B[1:]):
        X = v.split('v')
        for j in X:
            f.write('{j}'+',')
        f.write('\n')
print("fin :)")

풀이해설

In [2]:
import requests
import json
import datetime as dt
import concurrent.futures as futures
from tqdm.notebook import tqdm

url = 'https://data.hossam.kr/py/bus_station.csv'
session = requests.Session()

session.headers.update({
    "Referer" : "",
    "User_Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
})
try: 
    r = session.get(url)
    if r.status_code != 200:
        msg = f'[{r.status_code} Error] {r.reason} 에러가 발생함'
        raise Exception(msg)
except Exception as e :
    print('접속에 실패했습니다.', e, sep = '\n')
r.encoding = 'euc-kr' # 읽은 후 한글이 깨지면 euc-kr로 변경
# 정류소번호,	정류소명,	X좌표,	Y좌표,	정류소 타입  + 우편번호 + 주소
# print(r.text)
A = r.text
B = A.split('\n')

# 브이월드 인증키 : DB986055-D8D7-3F8F-9BBE-3F4E00712330

def SetZipcodeandAddress(a,b,session):
        url = 'https://api.vworld.kr/req/address?service=address&request=getAddress&key=DB986055-D8D7-3F8F-9BBE-3F4E00712330&point=%s,%s&type=both' % (a,b)
        try: 
            r = session.get(url)
            if r.status_code != 200:
                msg = f'[{r.status_code} Error] {r.reason} 에러가 발생함'
                raise Exception(msg)
        
            r.encoding = 'utf-8' # 읽은 후 한글이 깨지면 euc-kr로 변경
            mydict = json.loads(r.text)
        except Exception as e :
            print('접속에 실패했습니다.', e, sep = '\n')

        return [mydict['response']['result'][0]['zipcode'],mydict['response']['result'][0]['text']]
# 함수안에서 리스트나 딕셔너리 사용시 해당리스트/딕셔너리는 참조(포인터)이므로 따로 리턴하지 않아도 함수 외부에서도 해당 결과가 동일하게 적용된다.

dataset = []
# 프로그래스 바 객체 생성 
progress = tqdm(total=len(B)-1)


with futures.ThreadPoolExecutor(max_workers=50) as executor:
    for i, v in enumerate(B[1:]):
        if v == '':break
        C = v.split(',')
        try:
            a = float(C[2])
            b = float(C[3])
        except:
            a = float(C[3])
            b = float(C[4])
        
        # 비동기 처리 객체를 submit() 메서드로부터 리턴받는다.
        try:
            fu = executor.submit(SetZipcodeandAddress,a,b,session)
            
            # 현재 실행중인 함수(queryAddress)가 리턴하는 값을 반환받는다.
            result = fu.result()  
            dataset.append(C+result)
        except:
            pass
        # 비동기 작업이 끝났을 때 콜백함수를 호출한다.
        fu.add_done_callback(lambda x : progress.update())



fname = dt.datetime.now().strftime('주소조회_%y%m%d_%H%M%S.csv')
with open(fname,'w',encoding='utf-8') as f:
    f.write('정류소번호, 정류소명, X좌표, Y좌표, 정류소 타입, 우편번호, 주소\n')
    for i in dataset:
        f.write(f"{i}")
        f.write("\n")
        
print("fin :)")


  0%|          | 0/11291 [00:00<?, ?it/s]

fin :)
